# Q1
* Who did spend most money for renting?

In [1]:
import operator
mostpay = Customer.objects.all().annotate(total=Sum('rent__cost')).latest('total')
print ('ID=' + str(mostpay.id) + ': '+ mostpay.first_name + ' ' + mostpay.last_name + ' ' + str(mostpay.total))

ID=2: Ronald Cunningham 1799.95


# Q2
* Which car does make the most amount of income?

In [2]:
mostincome = Car.objects.annotate(total=Sum('rent__cost')).latest('total')
print ('ID=' + str(mostincome.id) + ': ' + mostincome.model, str(mostincome.total))

ID=3: Tuscan 2240.00


# Q3 
* How many time Erick Gilbert (id=6) rent the cars? (need to query from the name)

In [3]:
countrent = Customer.objects.annotate(Count('rent')).get(first_name='Erick', last_name='Gilbert', id=6)#Erick Gilbert has multiple object
print (countrent.rent__count)

3


# Q4
* Q4 What are the cars rented by Ronald Cunningham? (need to query from the name

In [4]:
rentcars = Customer.objects.get(first_name='Ronald', last_name='Cunningham').rent_set.all()
for rentcar in rentcars.iterator():
    print ('ID=' + str(rentcar.car.id) + ': ' + rentcar.car.model)

ID=2: Cooper
ID=5: Celica
ID=6: TT
ID=1: L200


# Q5
* Between 17/03/2014(inclusive) and 29/03/2014(inclusive), what is the total income of ALL cars in the range of time? (use return date)

In [5]:
from datetime import datetime
import pytz
utc=pytz.timezone('UTC')
start_date = utc.localize( datetime.strptime('2014-03-13','%Y-%m-%d') )
stop_date = utc.localize( datetime.strptime('2014-03-24','%Y-%m-%d') )

In [6]:
sumrentintime = Rent.objects.filter(rent_date__range=[start_date, stop_date]).aggregate(Sum('cost'))
print (sumrentintime['cost__sum'])

3309.50


# Q6
* Between 17/03/2014(inclusive) and 29/03/2014(inclusive), (use return date) what are the total incomes of EACH car in the range of time?

In [7]:
#use date from Q5
sumrentintimebycar = Rent.objects.filter(rent_date__range=[start_date, stop_date]).annotate(Sum('cost'))
for sumrent in sumrentintimebycar.iterator():
    print ('carID='+str(sumrent.car.id), sumrent.cost__sum)

carID=3 1310.00
carID=2 425.00
carID=4 189.95
carID=5 50.00
carID=6 269.95
carID=7 514.85
carID=1 549.75


# Q7
* Find all rental serveices that has duration less than than 3 days?

In [8]:
from django.db.models import F, ExpressionWrapper, fields
from datetime import timedelta
duration_cal = ExpressionWrapper(F('return_date') - F('rent_date'), output_field=fields.DurationField())
rents3day = Rent.objects.all().annotate(duration=duration_cal).filter(duration__lte=timedelta(days=3))
for rent3day in rents3day:
    print ('rent_id='+str(rent3day.id), rent3day.duration)

rent_id=4 3 days, 0:00:00
rent_id=5 1 day, 0:00:00
rent_id=6 0:00:00
rent_id=7 1 day, 0:00:00
rent_id=8 3 days, 0:00:00
rent_id=10 0:00:00
rent_id=11 0:00:00
rent_id=12 0:00:00
rent_id=13 -1 day, 0:00:00
rent_id=14 -1 day, 0:00:00
rent_id=15 -1 day, 0:00:00
rent_id=16 -1 day, 0:00:00
rent_id=17 -1 day, 0:00:00


# Q8
* Find all rental serveices that has duration greater than or equal than 2 day?

In [9]:
#duration_cal from Q7
more2day = Rent.objects.all().annotate(duration=duration_cal).filter(duration__gte=timedelta(days=2))
for i in more2day:
    print ('rentID='+str(i.id), i.duration)

rentID=1 5 days, 0:00:00
rentID=2 8 days, 0:00:00
rentID=3 7 days, 0:00:00
rentID=4 3 days, 0:00:00
rentID=8 3 days, 0:00:00
rentID=9 5 days, 0:00:00


# Q9
* Find the total rent duration of EACH car?

In [10]:
duration_cal = ExpressionWrapper(F('rent__return_date') - F('rent__rent_date'), output_field=fields.DurationField())
totalrentpercars = Car.objects.all().annotate(duration_cap=Sum(duration_cal))
for totalrentpercar in totalrentpercars:
    print ('carID='+str(totalrentpercar.id),totalrentpercar.duration_cap)

carID=1 9 days, 0:00:00
carID=2 11 days, 0:00:00
carID=3 6 days, 0:00:00
carID=4 0:00:00
carID=5 -1 day, 0:00:00
carID=6 0:00:00
carID=7 3 days, 0:00:00


# Q10
* Find the ratio [total_income_of_car]/[car_price] of EACH car?

In [11]:
incomeratios = Car.objects.all().annotate(income=Sum('rent__cost')/F('price'))
for incomeratio in incomeratios:
    print ('carID=' + str(incomeratio.id), incomeratio.income)

carID=1 0.15
carID=2 0.00
carID=3 0.00
carID=4 0.08
carID=5 0.00
carID=6 0.03
carID=7 0.03
